# SEIRX model runs for prevention measures investigation in universities

In [35]:
import pandas as pd
import numpy as np
from os.path import join
import os
import sys
import json

from scseirx import analysis_functions as af

import data_creation_functions as dcf

# parallelisation functionality
from multiprocess import Pool
import psutil
from tqdm import tqdm

from importlib import reload

In [8]:
# basic prevention measures in place. Additional measures will be added
# throughout the measure screening simulations
with open('params/intervention_screening_measures.json', 'r') as fp:
    measures = json.load(fp)
# simulation parameters, specifically the
# - base transmission risk (calibrated from household transmissions)
# - subclinical transmission modifier (literature value)
# - exposure duration, time until symtpoms and infection duration (lit. values)
# - age symptom discount (fit to empirical observations)
# - age transmission discount (from the calibration)
# - weights of transmission risks for intermediate & far contacts (calibration)
with open('params/intervention_screening_simulation_parameters.json', 'r') as fp:
    simulation_params = json.load(fp)

In [9]:
measures

{'testing': 'preventive',
 'preventive_screening_test_type': 'same_day_antigen',
 'diagnostic_test_type': 'two_day_PCR',
 'K1_contact_types': ['close'],
 'quarantine_duration': 10,
 'unistudent_mask': None,
 'lecturer_mask': None,
 'unistudent_screen_interval': None,
 'lecturer_screen_interval': None,
 'follow_up_testing_interval': None,
 'liberating_testing': False,
 'transmission_risk_ventilation_modifier': None,
 'unistudent_vaccination_ratio': 0.8,
 'lecturer_vaccination_ratio': 0.7}

In [10]:
simulation_params

{'verbosity': 0,
 'base_transmission_risk': 0.16598,
 'subclinical_modifier': 0.6,
 'exposure_duration': [3.0, 1.9],
 'time_until_symptoms': [4.4, 1.9],
 'infection_duration': [10.91, 3.95],
 'unistudent_index_probability': 0.9319711343549826,
 'lecturer_index_probability': 0.06802886564501744,
 'infection_risk_contact_type_weights': {'very_far': 0,
  'far': 0.26,
  'intermediate': 0.26,
  'close': 1},
 'age_transmission_discount': {'slope': 0, 'intercept': 1},
 'age_symptom_discount': {'slope': 0, 'intercept': 0.6},
 'mask_filter_efficiency': {'exhale': 0.5, 'inhale': 0.7},
 'vaccination_modification': {'reception': 0.6, 'transmission': 0.0}}

In [81]:
# number of runs in the ensemble (set to a low number so runs complete quickly
# for demonstration purposes)
N_runs = 14

# load the other screening parameters from file
screening_params = pd.read_csv(join('screening_params', 'measure_packages.csv'))

params = [(N_runs, 
           row['u_mask'],
           row['l_mask'], 
           row['presence_fraction'])
           for i, row in screening_params.iterrows()]

print('there are {} different parameter combinations'.format(len(params)))

there are 12 different parameter combinations


In [82]:
params[-2:-1]

[(14, True, False, 0.25)]

In [84]:
%%time
reload(dcf)
reload(af)

# paths for data I/O
contact_network_src = '../data/networks'
dst = '../data/simulation_results/ensembles'

results = pd.DataFrame()
for p in params[-2:-1]:
    N_runs, unistudent_mask, lecturer_mask, presence_fraction = p
    
    dcf.run_ensemble(N_runs, measures,\
            simulation_params, contact_network_src, dst, 
            unistudent_mask=unistudent_mask, lecturer_mask=lecturer_mask,
            presence_fraction=presence_fraction, testing=False)

running on T14s, using 14 cores


100%|██████████| 14/14 [09:20<00:00, 40.07s/it]


CPU times: user 8min 44s, sys: 7.99 s, total: 8min 52s
Wall time: 9min 26s
